In [ ]:
pip install tensorflowjs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.1/89.1 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 2.4 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.2
    Uninstalling packaging-24.2:
      Successfully uninstalled packaging-24.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 3.5.0 requires fsspec[http]<=2024.12.0,>=2023.1.0, but you have fsspec 2025.3.2 which is incompatible.
nilearn 0.11.1 requires scikit-learn>=1.4.0, but you have scikit-learn 1.2.2 which is incompatible.
pandas-gbq 0.26.1 requires google-api-core<3.0.0dev,>=2.10.2, but you have google-api-core 1.34.1 which is incompatible.
bigframes 1.36.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.
plotnine 0.14.5 requires matplotlib>=3.8.0, but you have matplotlib 3.7.5 which is 

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import os

2025-05-24 22:22:02.777456: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748125323.039521      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748125323.116937      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
data = pd.read_csv('datasetfinal.csv')
data

,seller_id,product_name,product_id,buyer_id,product_rating,product_price
0,457,40 Set Menu Sehari-hari Hits di Instagram ala ...,30070,1504,4.9,142800
1,457,7 Hari Belajar Drone Photography (Edisi Revisi),30079,1435,5.0,102000
2,457,Akasha : City Hunter - Complete Edition 03,30121,1560,4.9,49300
3,457,AKASHA : FATE/APOCRYPHA 02,30163,271,4.8,38250
4,457,AKASHA : RECORD OF RAGNAROK 03,30219,6,5.0,38250
...,...,...,...,...,...,...
49995,105,Samyang V-AF 24mm T1.9 Lens for Sony FE Samyan...,23658,47,5.0,8979000
49996,135,Godox S60 LED Focusing 3-Light Kit,29682,132,5.0,18849000
49997,134,Memory 128GB SF-M Tough Series UHS-II SDXC Mem...,28802,250,5.0,1599000
49998,4,Insta360 One X3 Sticky Lensguard,20253,248,5.0,539000


In [ ]:
def preprocess_data(data):
    # Drop baris dengan product_rating == 'No Rating'
    df = data[data['product_rating'] != 'No Rating'].copy()

    # Konversi ke float
    df['product_rating'] = df['product_rating'].astype(float)

    # Normalize price and rating
    scaler = MinMaxScaler()
    df[['norm_price', 'norm_rating']] = scaler.fit_transform(df[['product_price', 'product_rating']])

    # TF-IDF untuk nama produk
    tfidf = TfidfVectorizer(stop_words='english')
    tfidf_matrix = tfidf.fit_transform(df['product_name'])

    # Gabungkan semua fitur ke dalam item vector
    price_rating_features = df[['norm_price', 'norm_rating']].values
    combined_features = np.hstack([tfidf_matrix.toarray(), price_rating_features])

    return df, combined_features

In [ ]:
df, combined_features = preprocess_data(data)

In [ ]:
# Simpan untuk inference
product_ids = df['product_id'].values
product_id_to_index = {pid: idx for idx, pid in enumerate(product_ids)}
index_to_product_id = {idx: pid for pid, idx in product_id_to_index.items()}

# Modelling

In [ ]:
# Buat model embedding sederhana (dense layer)
input_dim = combined_features.shape[1]
output_dim = 64

# Buat model autoencoder
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(input_dim,)),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(output_dim, activation='linear'),  # 64 dimensi
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(input_dim, activation='linear')  # Output size sama dengan input
])

# Compile dan train
model.compile(optimizer='adam', loss='mse')
model.fit(combined_features, combined_features, epochs=100, batch_size=64, verbose=1)

2025-05-24 22:22:20.880866: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Epoch 1/100
775/775 ━━━━━━━━━━━━━━━━━━━━ 60s 75ms/step - loss: 1.0991e-04
Epoch 2/100
775/775 ━━━━━━━━━━━━━━━━━━━━ 53s 68ms/step - loss: 9.3611e-05
Epoch 3/100
775/775 ━━━━━━━━━━━━━━━━━━━━ 52s 67ms/step - loss: 9.2444e-05
Epoch 4/100
775/775 ━━━━━━━━━━━━━━━━━━━━ 53s 68ms/step - loss: 9.0033e-05
Epoch 5/100
775/775 ━━━━━━━━━━━━━━━━━━━━ 51s 65ms/step - loss: 8.7989e-05
Epoch 6/100
775/775 ━━━━━━━━━━━━━━━━━━━━ 54s 69ms/step - loss: 8.6148e-05
Epoch 7/100
775/775 ━━━━━━━━━━━━━━━━━━━━ 78s 63ms/step - loss: 8.4483e-05
Epoch 8/100
775/775 ━━━━━━━━━━━━━━━━━━━━ 53s 68ms/step - loss: 8.2788e-05
Epoch 9/100
775/775 ━━━━━━━━━━━━━━━━━━━━ 51s 66ms/step - loss: 8.1034e-05
Epoch 10/100
775/775 ━━━━━━━━━━━━━━━━━━━━ 50s 65ms/step - loss: 7.9351e-05
Epoch 11/100
775/775 ━━━━━━━━━━━━━━━━━━━━ 49s 63ms/step - loss: 7.7792e-05
Epoch 12/100
775/775 ━━━━━━━━━━━━━━━━━━━━ 51s 65ms/step - loss: 7.6233e-05
Epoch 13/100
775/775 ━━━━━━━━━━━━━━━━━━━━ 82s 66ms/step - loss: 7.4558e-05
Epoch 14/100
775/775 ━━━━━━━━━━━━━

In [ ]:
import os
import tensorflow as tf

# Save the model in TensorFlow's SavedModel format
save_path = "/kaggle/working/recommendation_model"  # Just directory path, no extension
tf.saved_model.save(model, save_path)  # Use tf.saved_model.save() for SavedModel format
print("✅ Saved TensorFlow model in SavedModel format")

# Convert to TFJS (TensorFlow.js) format
os.system(f"tensorflowjs_converter --input_format=tf_saved_model --output_format=tfjs_graph_model {save_path} /kaggle/working/tfjs_model/")
print("✅ Converted to TensorFlow.js format")

# Save model in HDF5 (.h5) format
model.save("/kaggle/working/recommendation_model.h5")
print("✅ Saved model in HDF5 (.h5) format")


✅ Saved TensorFlow model in SavedModel format


2025-05-24 23:56:13.438229: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748130973.455866     869 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748130973.461065     869 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
🌲 Try https://ydf.readthedocs.io, the successor of TensorFlow Decision Forests with more features and faster training!
2025-05-24 23:56:19.944033: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
I0000 00:00:1748130980.281053     869 devices.cc:67] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
I0000 00:00

✅ Converted to TensorFlow.js format
✅ Saved model in HDF5 (.h5) format


# Inference

In [ ]:
def recommend(product_id, top_n=20):
    if product_id not in product_id_to_index:
        print("Product ID not found.")
        return []

    query_index = product_id_to_index[product_id]

    # Embed all items
    item_embeddings = model.predict(combined_features, verbose=0)
    query_embedding = item_embeddings[query_index].reshape(1, -1)

    # Compute cosine similarity
    similarities = cosine_similarity(query_embedding, item_embeddings)[0]

    # Ambil top N rekomendasi selain dirinya sendiri
    similar_indices = similarities.argsort()[::-1]
    similar_indices = [i for i in similar_indices if i != query_index][:top_n]

    recommended_df = df.iloc[similar_indices].copy()
    recommended_df['similarity'] = similarities[similar_indices]

    return recommended_df

# Contoh: rekomendasi untuk product_id 10001
recommendations = recommend(10001)
pd.set_option('display.max_rows', None)
print(recommendations)

       seller_id                                       product_name  \
30115        593  Cangkir + Tutup Seri 10 Ayam Jago Seri 2 / Mug...   
30123        593  Cangkir + Tutup Seri 6 Ayam Jago Seri 2 - Cang...   
30124        593  Cangkir + Tutup Seri 5 Ayam Jago Seri 2 - Cang...   
30489        804  Gelas Cangkir Seri 10 Ayam Jago Seri 2 - Mug T...   
30030       1311  6 pasang Cangkir Set Seri 6 Ayam Jago seri 2 -...   
30171       1337  6 Pasang Cangkir Set Seri 8 Ayam Jago Seri 2 /...   
30142        593  6 Pasang Cangkir Set seri 3. Ayam Jago seri 2 ...   
30135        593  6 Pasang Cangkir Set Seri 2 Ayam Jago Seri 2 -...   
30163       1337  6 pasang Cangkir Set seri 7 Ayam Jago seri 2 -...   
30154       1337  2 Pcs Cangkir Tutup + Asbak AYAM JAGO SERI 2 /...   
30389        579  Satu Pasang - Cangkir Kopi & Saucer Seri 3 Aya...   
30428       1062  Satu Pasang - Cangkir Kopi & Saucer Seri 2 Aya...   
30350       1668  Satu Pasang - Cangkir Kopi & Saucer Seri 7 Aya...   
30129 